In [1]:
# Set up spark
import spark_session_builder
import spark_session_builder

sparkSession, sc = spark_session_builder.session()
sc

<SparkContext master=local[*] appName=testing>

In [2]:
# Key imports

from pyspark.sql.functions import col, split, explode
import pyspark.sql.functions as f

from PWBdata import PWBdata
hdfs_path = "hdfs://demo-full-load-hdfs-namenode:8020"

## Load the Compounds Mapping File

In [3]:
filename = 'compounds.csv' + '/*.snappy.parquet'
df = sparkSession \
    .read \
    .option("header", "true") \
    .parquet(filename)

In [4]:
from IPython.core.display import HTML


def show_top(df, records=10):
    """
    display the top n records in a table
    """
    display(HTML(df.limit(records).toPandas().to_html(escape=False)))

## Query the Compounds data for a single drug

In [5]:
query = 'oxytocin'
query_df = df.filter(col('hasPreferredMoleculeName') == query.upper())
show_top(query_df)

,CHEMBL_ID,fingerprint,reaxys_chemicalcompound,resnet_smallmol,pharmapendium_drug,Compound,hasPreferredMoleculeName
0,CHEMBL395429,iri:uniprot/CHEMBL395429,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/17516222,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038355159,https://data.elsevier.com/lifescience/entity/ppplus/drug/iNXuLnsxtf_,https://data.elsevier.com/lifescience/entity/chembl/compound/392195,OXYTOCIN
1,CHEMBL395429,iri:uniprot/CHEMBL395429,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/15430736,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594039151119,https://data.elsevier.com/lifescience/entity/ppplus/drug/iNXuLnsxtf_,https://data.elsevier.com/lifescience/entity/chembl/compound/392195,OXYTOCIN
2,CHEMBL395429,iri:uniprot/CHEMBL395429,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/6048494,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594039151119,https://data.elsevier.com/lifescience/entity/ppplus/drug/iNXuLnsxtf_,https://data.elsevier.com/lifescience/entity/chembl/compound/392195,OXYTOCIN
3,CHEMBL395429,iri:uniprot/CHEMBL395429,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/5709910,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594039151119,https://data.elsevier.com/lifescience/entity/ppplus/drug/iNXuLnsxtf_,https://data.elsevier.com/lifescience/entity/chembl/compound/392195,OXYTOCIN
4,CHEMBL395429,iri:uniprot/CHEMBL395429,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/10145454,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038180209,https://data.elsevier.com/lifescience/entity/ppplus/drug/iNXuLnsxtf_,https://data.elsevier.com/lifescience/entity/chembl/compound/392195,OXYTOCIN
5,CHEMBL395429,iri:uniprot/CHEMBL395429,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/5709910,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038180209,https://data.elsevier.com/lifescience/entity/ppplus/drug/iNXuLnsxtf_,https://data.elsevier.com/lifescience/entity/chembl/compound/392195,OXYTOCIN
6,CHEMBL395429,iri:uniprot/CHEMBL395429,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/24162959,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038381587,https://data.elsevier.com/lifescience/entity/ppplus/drug/iNXuLnsxtf_,https://data.elsevier.com/lifescience/entity/chembl/compound/392195,OXYTOCIN
7,CHEMBL395429,iri:uniprot/CHEMBL395429,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/10145454,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038381587,https://data.elsevier.com/lifescience/entity/ppplus/drug/iNXuLnsxtf_,https://data.elsevier.com/lifescience/entity/chembl/compound/392195,OXYTOCIN
8,CHEMBL395429,iri:uniprot/CHEMBL395429,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/17516222,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038180209,https://data.elsevier.com/lifescience/entity/ppplus/drug/iNXuLnsxtf_,https://data.elsevier.com/lifescience/entity/chembl/compound/392195,OXYTOCIN
9,CHEMBL395429,iri:uniprot/CHEMBL395429,https://data.elsevier.com/lifescience/entity/reaxys/chemicalcompound/6048494,https://data.elsevier.com/lifescience/entity/resnet/smallmol/72057594038180209,https://data.elsevier.com/lifescience/entity/ppplus/drug/iNXuLnsxtf_,https://data.elsevier.com/lifescience/entity/chembl/compound/392195,OXYTOCIN


## What does RMC know about this drug?

In [6]:
pwb = PWBdata(hdfs_path)

In [7]:
#  Chemical Compounds are linked Compound-AgentConfiguration-BiologicalActivity-MeasureSettings
# AGENT
agent_df = pwb.get_dataframe(sparkSession, 'rmc', 'agentconfiguration').select('hasChemicalCompound', 'AgentConfiguration')
agent_for_query = query_df.join(agent_df, query_df.reaxys_chemicalcompound == agent_df.hasChemicalCompound)
#show_top(agent_for_query)

In [8]:
# BIOLOGICAL ACTIVITY
bioact_df = pwb.get_dataframe(sparkSession, 'rmc', 'biologicalactivity').select('BiologicalActivity','hasMeasureSettings','hasAgentConfiguration')
bioact_for_query = agent_for_query.join(bioact_df, agent_for_query.AgentConfiguration == bioact_df.hasAgentConfiguration)
#show_top(bioact_for_query)

In [9]:
# MEASURE SETTINGS
measure_df = pwb.get_dataframe(sparkSession, 'rmc', 'measuresettings') \
    .withColumn("hasBiologicalActivity", explode(split("hasBiologicalActivity", "\|\|"))) \
    .select('MeasureSettings', 'hasMeasuredParameter', 'hasBioAssay', 'hasChemicalCompound', 'hasBiologicalActivity') \
    .withColumnRenamed('hasBiologicalActivity', 'BiologicalActivity')
parameter_df = pwb.get_dataframe(sparkSession, 'rmc', 'measuredparameter').select('MeasuredParameter', 'hasLabel').withColumnRenamed('hasLabel', 'parameter')
measure_with_labels = measure_df.join(parameter_df, parameter_df.MeasuredParameter == measure_df.hasMeasuredParameter)
measure_for_query = bioact_for_query.join(measure_with_labels, 'BiologicalActivity')
#show_top(measure_for_query)

In [10]:
show_top(measure_for_query.groupBy('parameter').count().orderBy('count', ascending=False), 20)

,parameter,count
0,stimulation percentage,68
1,EC50,68
2,qualitative,48
3,Emax,48
4,pEC50,20
5,Ki (inhibition constant),20
6,IC50,16
7,t1/2 el,8
8,pKi,8
9,GI,8


## What does Pharmapendium know about this drug?

In [11]:
# drugs link Dosing Regimen-Study-StudyDesign
# DOSING REGIMEN
pp_dosing_regimen_df = pwb.get_dataframe(sparkSession, 'pp', 'Dosingregimen').select('DosingRegimen', 'hasDrug' ).withColumnRenamed('hasDrug', 'pharmapendium_drug')
query2regimen_df = query_df.join(pp_dosing_regimen_df, 'pharmapendium_drug')
#  STUDY
pp_study_df = pwb.get_dataframe(sparkSession, 'pp', 'Study').select('Study','hasDosingRegimen', 'hasMeasure', 'hasStudyDesign') \
    .withColumnRenamed('hasDosingRegimen', 'DosingRegimen') \
    .withColumnRenamed('hasStudyDesign', 'StudyDesign')
query2study = query2regimen_df.join(pp_study_df, 'DosingRegimen')
# show_top(query2study)

In [12]:
# STUDY DESIGN
pp_studydesign_df = pwb.get_dataframe(sparkSession, 'pp', 'Studydesign').select('StudyDesign','hasStudyDesignType') \
    .withColumnRenamed('hasStudyDesignType','StudyDesignType')
pp_studydesigntype_df = pwb.get_dataframe(sparkSession, 'pp', 'Studydesigntype').select('StudyDesignType','core_prefLabel') \
    .withColumnRenamed('core_prefLabel', 'module')
pp_studydesign_with_labels = pp_studydesign_df.join(pp_studydesigntype_df, 'StudyDesignType')
# show_top(pp_studydesign_with_labels)

In [13]:
pp_results_df = query2study.join(pp_studydesign_with_labels, 'StudyDesign').groupBy('module').count()
show_top(pp_results_df)

,module,count
0,safety,5376
1,faers,55384
2,pk,280
3,me,560
4,activity,15568


## What does Resnet know about this drug?

In [63]:
# is regulator of
resnet4reg_df = pwb.get_dataframe(sparkSession, 'resnet', 'Smallmol').select('SmallMol','isRegulatorOf') \
    .withColumnRenamed('SmallMol', 'resnet_smallmol')
regulated_df = resnet4reg_df.withColumn("Regulates", explode(split("isRegulatorOf", "\|\|"))).drop('isRegulatorOf')
regulated4query = query_df.join(regulated_df, 'resnet_smallmol')
regulator_count = regulated4query.select("Regulates").distinct().count()
regulators = regulated4query.select("Regulates").distinct()
reg_types  = regulators.withColumn('regulating_entity', f.split(f.col("Regulates"),"/").getItem(6))
show_top(reg_types.groupBy('regulating_entity').count())

,regulating_entity,count
0,clinicaltrial,2
1,directregulation,6
2,regulation,7


In [65]:
# is partner of
resnet4partner_df = pwb.get_dataframe(sparkSession, 'resnet', 'Smallmol').select('SmallMol','isPartnerOf') \
    .withColumnRenamed('SmallMol', 'resnet_smallmol')
partnered_df = resnet4partner_df.withColumn("Partners", explode(split("isPartnerOf", "\|\|"))).drop('isPartnerOf')
partnered4query = query_df.join(partnered_df, 'resnet_smallmol')
partner_count = partnered4query.select("Partners").distinct().count()
partners = partnered4query.select("Partners").distinct()
partner_types  = partners.withColumn('partnering_entity', f.split(f.col("Partners"),"/").getItem(6))
show_top(partner_types.groupBy('partnering_entity').count())

,partnering_entity,count
0,binding,2


In [66]:
# is target of
resnet4target_df = pwb.get_dataframe(sparkSession, 'resnet', 'Smallmol').select('SmallMol','isTargetOf') \
    .withColumnRenamed('SmallMol', 'resnet_smallmol')
target_df = resnet4target_df.withColumn("Targets", explode(split("isTargetOf", "\|\|"))).drop('isTargetOf')
target4query = query_df.join(target_df, 'resnet_smallmol')
targets = target4query.select("Targets").distinct()
target_types  = targets.withColumn('targeting_entity', f.split(f.col("Targets"),"/").getItem(6))
show_top(target_types.groupBy('targeting_entity').count())

,targeting_entity,count
